AudioCraft github: https://github.com/facebookresearch/audiocraft  
AudioCraft API documentation: https://facebookresearch.github.io/audiocraft/api_docs/audiocraft/index.html  


AudioCraft - MusicGen: https://github.com/facebookresearch/audiocraft/blob/main/docs/MUSICGEN.md  

In [ ]:
# Deal with the bug ''NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968 '
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip show torch

Name: torch
Version: 2.0.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton


In [ ]:
base_path='/content/drive/MyDrive/Wire_Note_Project/try_audio_to_sheet'

In [ ]:
%cd f{base_path}
%pwd

In [ ]:
!pip install -U audiocraft

In [ ]:
import os
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

In [ ]:
output_path=os.path.join(base_path,'output')
os.makedirs(output_path, exist_ok=True)
os.path.exists(output_path)

True

Generate melody from video

In [ ]:
video_path=os.path.join(base_path,'test_audio.mp3')
os.path.exists(video_path)

True

In [ ]:
model = MusicGen.get_pretrained('facebook/musicgen-small')

print(model)

In [ ]:
melody_model = MusicGen.get_pretrained('facebook/musicgen-melody')

print(melody_model)

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 129MB/s]


In [ ]:
model.set_generation_params(duration=8)  # generate 8 seconds.
wav = model.generate_unconditional(4)    # generates 4 unconditional audio samples
descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
wav = model.generate(descriptions)  # generates 3 samples.

melody, sr = torchaudio.load(video_path)
# generates using the melody from the given audio and the provided descriptions.
wav = melody_model.generate_with_chroma(descriptions, melody[None].expand(3, -1, -1), sr)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)



Text to video

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-t7qyva9c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-t7qyva9c
  Resolved https://github.com/huggingface/transformers.git to commit 8e3980a290acc6d2f8ea76dba111b9ef0ef00309
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7700262 sha256=283a9b5386aa43244ce2d26ac43353769b04725618fa81bdeb165d593a54947c
  Stored in directory: /tmp/pip-ephem-wheel-cache-q85_aga4/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Fou

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

In [ ]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

text=['sad,very sad']
inputs = processor(
    text=text,
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=256)

In [ ]:
import scipy

output_path_1=os.path.join(output_path,"musicgen_out-2.wav")

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write(output_path_1, rate=sampling_rate, data=audio_values[0, 0].numpy())

In [ ]:
from google.colab import files

files.download(output_path_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>